In [155]:
"""
Step 1: Load Dataset
Step 2: Make Dataset Iterable
Step 3: Create Model Class
Step 4: Instantiate Model Class
Step 5: Instantiate Loss Class
Step 6: Instantiate Optimizer Class
Step 7: Train Model
https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_lstm_neuralnetwork/
"""
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as torch_data
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from data_loader.dataloader import LSTMTSD_Dataset
import numpy as np

In [ ]:
""" DONt CHANGE
Parameters for LSTMModel_v0
"""
RATIO_SPLIT = 0.8
batch_size = 200
n_iters = 10000000
seq_dim = 1
iter = 0
input_dim = 10
hidden_dim = 100
layer_dim = 1
output_dim = 3
learning_rate = 0.00001
horizon = 1
window_size= 5

In [ ]:
## Step 1: Load Dataset

path = './dataset/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우

## csv 파일들을 DataFrame으로 불러와서 concat

df = pd.DataFrame()

df = pd.read_csv("./dataset/Telegram_1hour_7.csv")
df.insert(2, "label", int(0))
df_0 = df[["Time", "Length", "label"]].to_numpy()

df = pd.read_csv("./dataset/Zoom_1hour_5.csv")
df.insert(2, "label", int(1))
df_1 = df[["Time", "Length", "label"]].to_numpy()

df = pd.read_csv("./dataset/YouTube_1hour_2.csv")
df.insert(2, "label", int(2))
df_2 = df[["Time", "Length", "label"]].to_numpy()

df_set = np.vstack((df_0, df_1, df_2))
df_set = LSTMTSD_Dataset(df_set, window_size=window_size, horizon=horizon, normalize_method="z_score")
#dataloader = DataLoader(df_set, batch_size=1, drop_last=False, shuffle=True, num_workers=0)


train_dataset, val_dataset = torch.utils.data.random_split(df_set, [int(len(df_set)*RATIO_SPLIT),len(df_set) - int(len(df_set)*RATIO_SPLIT)])
val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [int(len(val_dataset)*RATIO_SPLIT),len(val_dataset) - int(len(val_dataset)*RATIO_SPLIT)])

print("train_dataset:", len(train_dataset))
print("val_dataset:", len(val_dataset))
print("test_dataset:", len(test_dataset))

In [ ]:
## Step 2: Make Dataset Iterable

num_epochs = int(n_iters / (len(train_dataset) / batch_size))
#num_epochs = int(num_epochs)
#num_epochs = 100
print(int(num_epochs))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)
val_loader   = DataLoader(dataset=val_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)
test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)

In [ ]:
## Step 3: Create Model Class
class LSTMModel_v0(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel_v0, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
## Step 4: Instantiate Model Class
#model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model = LSTMModel_v0(input_dim, hidden_dim, layer_dim, output_dim)

## Step 5: Instantiate Loss Class
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

## Step 6: Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

len(list(model.parameters()))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
## Step 7: Train Model
# Number of steps to unroll

for epoch in range(num_epochs):
    for i, (inputs, labels, _) in enumerate(train_loader):
        inputs = inputs.view(-1, seq_dim, input_dim).requires_grad_()
        optimizer.zero_grad()
        outputs = model(inputs) #output: torch.Size([20, 1])
        loss = criterion(outputs, labels.type(torch.LongTensor))

        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1
        if iter % 50 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through  validation dataset
            for inputs, labels, _ in val_loader:
                inputs = inputs.view(-1, seq_dim, input_dim)
                outputs = model(inputs) #torch.Size([20, 1])
                _, predicted = torch.max(outputs.data, 1) #torch.Size([20])
                total += labels.size(0)
                predicted = predicted.resize(len(outputs), 1).type(torch.LongTensor) #torch.Size([20, 1])
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


In [ ]:
## CUDA

#  Returns a bool indicating if CUDA is currently available.
torch.cuda.is_available()
#  True

#  Returns the index of a currently selected device.
torch.cuda.current_device()
#  0

#  Returns the number of GPUs available.
torch.cuda.device_count()
#  1

#  Gets the name of a device.
torch.cuda.get_device_name(0)
#  'GeForce GTX 1060'

#  Context-manager that changes the selected device.
#  device (torch.device or int) – device index to select.
torch.cuda.device(0)
# Default CUDA device
cuda = torch.device('cuda')

iter = 0
input_dim = 10
hidden_dim = 100
layer_dim = 1
output_dim = 3
learning_rate = 0.00001
horizon = 1
window_size= 5

In [ ]:
import argparse
import torch

parser = argparse.ArgumentParser()
parser.add_argument('--cpu', action='store_true',help='run in cpu')
args = parser.parse_args()

if args.cpu:
    device = torch.device('cpu')
else:
    device = torch.device('cuda')

 x = torch.tensor([1., 2.]).to(device)

## Step 1: Load Dataset
df = pd.DataFrame()

df = pd.read_csv("./dataset/Telegram_1hour_7.csv")
df.insert(2, "label", int(0))
df_0 = df[["Time", "Length", "label"]].to_numpy()

df = pd.read_csv("./dataset/Zoom_1hour_5.csv")
df.insert(2, "label", int(1))
df_1 = df[["Time", "Length", "label"]].to_numpy()

df = pd.read_csv("./dataset/YouTube_1hour_2.csv")
df.insert(2, "label", int(2))
df_2 = df[["Time", "Length", "label"]].to_numpy()

df_set = np.vstack((df_0, df_1, df_2))
cuda = torch.device('cuda')
df_set = LSTMTSD_Dataset(df_set, window_size=window_size, horizon=horizon, normalize_method="z_score")
#dataloader = DataLoader(df_set, batch_size=1, drop_last=False, shuffle=True, num_workers=0)


train_dataset, val_dataset = torch.utils.data.random_split(df_set, [int(len(df_set)*RATIO_SPLIT),len(df_set) - int(len(df_set)*RATIO_SPLIT)])
val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [int(len(val_dataset)*RATIO_SPLIT),len(val_dataset) - int(len(val_dataset)*RATIO_SPLIT)])

print("train_dataset:", len(train_dataset))
print("val_dataset:", len(val_dataset))
print("test_dataset:", len(test_dataset))

In [ ]:
## Step 2: Make Dataset Iterable

num_epochs = int(n_iters / (len(train_dataset) / batch_size))
#num_epochs = int(num_epochs)
#num_epochs = 100
print(int(num_epochs))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)
val_loader   = DataLoader(dataset=val_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)
test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0)

In [ ]:
## Step 3: Create Model Class
class LSTMModel_v0(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel_v0, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
## Step 4: Instantiate Model Class
#model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model = LSTMModel_v0(input_dim, hidden_dim, layer_dim, output_dim)

## Step 5: Instantiate Loss Class
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

## Step 6: Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

len(list(model.parameters()))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
## Step 7: Train Model
# Number of steps to unroll

for epoch in range(num_epochs):
    for i, (inputs, labels, _) in enumerate(train_loader):
        inputs = inputs.view(-1, seq_dim, input_dim).requires_grad_()
        optimizer.zero_grad()
        outputs = model(inputs) #output: torch.Size([20, 1])
        loss = criterion(outputs, labels.type(torch.LongTensor))

        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1
        if iter % 50 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through  validation dataset
            for inputs, labels, _ in val_loader:
                inputs = inputs.view(-1, seq_dim, input_dim)
                outputs = model(inputs) #torch.Size([20, 1])
                _, predicted = torch.max(outputs.data, 1) #torch.Size([20])
                total += labels.size(0)
                predicted = predicted.resize(len(outputs), 1).type(torch.LongTensor) #torch.Size([20, 1])
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))